# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base: [Human Activity Recognition Using Smartphones](https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones). Por se tratar de uma base com um número elevado de variáveis, iremos realizar a redução de dimensionalidade.


In [223]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [225]:
#carregando as bases
filename_features = "features.txt"
filename_labels = "activity_labels.txt"

filename_subtrain = "subject_train.txt"
filename_xtrain = "X_train.txt"
filename_ytrain = "y_train.txt"

filename_subtest = "subject_test.txt"
filename_xtest = "X_test.txt"
filename_ytest = "y_test.txt"

# Lendo o arquivo de características e convertendo para lista
features = pd.read_csv(filename_features, header=None, sep="#")
features_list = features.iloc[:, 0].tolist()

# Lendo os rótulos
labels = pd.read_csv(filename_labels, sep=r'\s+', header=None, names=['cod_label', 'label'])

# Lendo os dados de treino
subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
X_train = pd.read_csv(filename_xtrain, sep=r'\s+', header=None, names=features_list)
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

# Lendo os dados de teste
subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
X_test = pd.read_csv(filename_xtest, sep=r'\s+', header=None, names=features_list)
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

Vamos observar o número de colunas da nossa base:

In [227]:
X_train.shape

(7352, 561)

Vimos aqui que nossa base dispõe de 561 colunas e ai vem um ponto importante, como definir as melhores variáveis para treinar um modelo? Afinal, quanto maior a base, maior o tempo de processamento. Para resolver esse problema utilizaremos a metodologia PCA (ANALISE DOS COMPONENTES PRINCIPAIS), que tem como objetivo fazer uma transformação linear nas variáveis através da sua correlação e criar então a quantidade de componentes necessária. É como se criassemos um novo data frame através da combinação das variáveis de um antigo.

Para iniciar a nossa análise e também comparação de tempo de processamento, iremos treinar uma árvore de ccp_alpha=0.001 para todas as variáveis, ou seja, sem alterar nosso df.

## Árvore de decisão

Iremos rodar uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. **Avaliaremos a acurácia** nas bases de treinamento e teste, assim como o tempo de processamento. Também **faremos a validação cruzada, calcularemos sua média e desvio padrão**.

In [234]:
%%time
# Criando e treinando a árvore de decisão
clf = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001).fit(X_train, y_train)

CPU times: total: 5.31 s
Wall time: 5.33 s


Agora que temos a árvore criada e treinada, iremos observar as **métricas de performance do nosso modelo**:

In [236]:
# Previsões
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

# Avaliando a acurácia
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Resultados
print(f"Acurácia na base de treino: {train_accuracy:.4f}")
print(f"Acurácia na base de teste: {test_accuracy:.4f}")

# Validação cruzada
cv_scores = cross_val_score(clf, X_train, y_train.values.ravel(), cv=5)  # 5-fold cross-validation

# Média e desvio padrão das acurácias
mean_cv_accuracy = cv_scores.mean()
std_cv_accuracy = np.std(cv_scores)

# Resultados
print(f"Acurácia média da validação cruzada: {mean_cv_accuracy:.4f}")
print(f"Desvio padrão da validação cruzada: {std_cv_accuracy:.4f}")

Acurácia na base de treino: 0.9758
Acurácia na base de teste: 0.8795
Acurácia média da validação cruzada: 0.8581
Desvio padrão da validação cruzada: 0.0309


**Desempenho Geral:** Juntas, essas métricas indicam que o modelo está se saindo bem, com uma boa capacidade de generalização, uma vez que a acurácia da base de teste e da validação cuzada indicam mais de 85% das previsões estão corretas. O modelo também apresenta um desvio padrão baixo, o que sugere que o modelo é consistente e confiável. 
_Quanto ao tempo_ tivemos um bom desempenho, uma vez que indicamos um ccp_alpha em vez de testar os obtidos pelo cost_complexity_pruning_path, mas vale lembrar que aqui temos mais de 500 variáveis, portanto, vamos observar como nosso modelo se sai após a redução de complexidade.


_obs_: no meu computador a árvore rodou em 5.33s. 

## Árvore com PCA  

### Apresentação das Funções Utilizadas:
Agora, iremos realizar a redução de complexidade, que ocorre quando o nosso modelo, através de uma transformação linear, utilizando todas as variáveis, criará o que chamamos de componentes principais, ou seja, novas variáveis que transmitem a correlação entre as variáveis originais. O número de novas variáveis, componentes desejadss deve ser informado.

Como nosso objetivo aqui é treinar árvores que recebem um número diferente de componentes principais, vamos agilizar o processo de criação de componentes principais, utilizando uma função que recebe como entrada a quantidade de componentes que desejamos criar e imprime o resultado.


In [241]:
def pca_qtd(X_train, X_test, n=1):
    # Ajustar PCA e transformar os dados
    prcomp = PCA(n_components=n).fit(X_train)
    pc_treino = prcomp.transform(X_train)
    pc_teste = prcomp.transform(X_test)

    # Criar nomes das colunas
    colunas = ['cp' + str(x + 1) for x in range(n)]

    # Criar DataFrames
    pc_train = pd.DataFrame(pc_treino[:, :n], columns=colunas)
    pc_test = pd.DataFrame(pc_teste[:, :n], columns=colunas)

    # Imprimir os DataFrames
    print("DataFrame de Treino:")
    print(pc_train.head(5))
    print("\nDataFrame de Teste:")
    print(pc_test.head(5))

Na hora de criar esses componentes principais, exibidos acima, todas as variáveis são utilizadas, entretanto, algumas tem uma influência muito maior que outras e **para descobrir quais são as 3 variáveis mais influentes na criação de um componente, vamos utilizar a função abaixo:**

_OBS_: utilizamos o método transform para obter os dados escalonados, transformados.

In [244]:
def significante_variavel_(df, n_components, top_n=3):
    # Padronizando os dados
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df)

    # Realizando PCA
    pca = PCA(n_components=n_components)
    pca.fit(X_scaled)

    # Obtendo os componentes principais
    components = pca.components_

    # Criando um DataFrame com os loadings
    loadings_df = pd.DataFrame(components, columns=df.columns, index=[f'PC{i+1}' for i in range(n_components)])

    # Criando uma lista para armazenar as informações para o DataFrame
    records = []

    for i in range(n_components):
        component_name = f'PC{i+1}'
        
        # Obtendo as variáveis e seus loadings: 
            #abs: Calcula o valor absoluto das cargas
            #nlargest(top_n): Retorna as top_n variáveis com maiores cargas absolutas.
        influential_vars = loadings_df.loc[component_name].abs().nlargest(top_n)
        
        for var, loading_value in influential_vars.items():
            records.append({'Component': component_name, 'Variable': var, 'Loading': loading_value})

    # Criando o DataFrame final
    organized_df = pd.DataFrame(records)

    return organized_df

A função acima será utilizada posteriormente e ela nos retornará as variáveis mais importantes de forma decrescente, lembrando que a função acima é mais para sanar curiosidade. Agora que já temos uma função para definir a quantidade de componentes principais e uma que nos mostra as variáveis mais influentes na composição destes, vamos treinar uma árvore de decisão com ccp_alpha = 0.001 e fazer a exibição das métricas.
O objetivo é treinar uma árvore semelhante àuela treinada antes de fazer a redução de complexidade do dataframe para comparar os resultados. Como serão várias árvores, criaremos uma **função para obter as métricas de maneira mais rápida**. 

In [125]:
def metricas_avaliacao(model, pc_treino, y_train, pc_teste, y_test, cv=5):
    # Criando DataFrames para as componentes principais
    colunas = ['cp' + str(x + 1) for x in range(pc_treino.shape[1])]
    pc_train = pd.DataFrame(pc_treino, columns=colunas)
    pc_test = pd.DataFrame(pc_teste, columns=colunas)

    # Treinando o modelo com todos os dados de treino
    model.fit(pc_train, y_train)

    # Acurácia na base de treino
    train_accuracy = accuracy_score(y_train, model.predict(pc_train))

    # Acurácia na base de teste
    test_accuracy = accuracy_score(y_test, model.predict(pc_test))

    # Validação cruzada
    cv_scores = cross_val_score(model, pc_train, y_train, cv=cv)
    mean_cv_accuracy = cv_scores.mean()
    std_cv_accuracy = np.std(cv_scores)

    # Resultados com arredondamento
    results = {
        "Acurácia na base de treino": round(train_accuracy, 3),
        "Acurácia na base de teste": round(test_accuracy, 3),
        "Acurácia média da validação cruzada": round(mean_cv_accuracy, 3),
        "Desvio padrão da validação cruzada": round(std_cv_accuracy, 3)
    }

    return results

Agora que criamos algumas funções para facilitar nossas análises, vamos inciar o treinamento da árvore de ccp_alpha=0.01 através da variação do número de componentes gerados. Como já criamos as funções, basta aplicá-las.

### Realização da Análise dos Componentes Principais e Treinamento de Modelo:

#### Transformação do dataframe e obtenção de um componente:

In [290]:
%%time
#transformação do df para 1 componente:
prcomp = PCA(n_components=1).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

CPU times: total: 469 ms
Wall time: 124 ms


In [281]:
#exibindo os componentes criados:
df_1=pca_qtd(X_train, X_test, n=1)

DataFrame de Treino:
        cp1
0 -5.520280
1 -5.535350
2 -5.474988
3 -5.677232
4 -5.748749

DataFrame de Teste:
        cp1
0 -2.686743
1 -4.331255
2 -4.985360
3 -5.099876
4 -5.023000


Vamos descobrir as 3 mais significativas utilizadas na criação do componente, vejamos para X_train.

In [259]:
significante_variavel_(X_train, 1, top_n=3)

,Component,Variable,Loading
0,PC1,281 fBodyAcc-sma(),0.058571
1,PC1,360 fBodyAccJerk-sma(),0.058568
2,PC1,96 tBodyAccJerk-sma(),0.058538


Agora que criamos o componente e vimos quais as principais variáveis utilizadas para sua transformação, vejamos as métricas de desempenho de uma árvore com ccp_alpha = 0.001 rodando esse novo df com apenas um componente.

In [262]:
%%time
clf1 = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001).fit(pc_treino, y_train)

CPU times: total: 31.2 ms
Wall time: 39.5 ms


In [264]:
metricas_avaliacao(clf1, pc_treino, y_train, pc_teste, y_test)

{'Acurácia na base de treino': 0.5,
 'Acurácia na base de teste': 0.457,
 'Acurácia média da validação cruzada': 0.475,
 'Desvio padrão da validação cruzada': 0.043}

O tempo de processamento da nossa árvore com um componente foi de 39,5ms, ou seja, quase 135 vezes mais rápido que a mesma árvore rodando com as 561 variáveis originais, logo, o processamento é muito mais rápido. Entretanto, como aqui temos apenas 1 componente, nossa acurácia é muito inferior à outra. Concluindo, temos um ganho enorme em tempo mas uma perca muito grande em qualidade. Vamos testar a mesma árvore mas agora com mais componentes gerados a partir dos 561 originais.

#### Transformação do dataframe e obtenção de dois componentes:

In [310]:
%%time
#transformação do df para 2 componentes:
prcomp = PCA(n_components=2).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

CPU times: total: 422 ms
Wall time: 125 ms


In [312]:
#exibindo os componentes:
df2= pca_qtd(X_train, X_test, n=2)
df2

DataFrame de Treino:
        cp1       cp2
0 -5.520280 -0.290278
1 -5.535350 -0.082530
2 -5.474988  0.287387
3 -5.677232  0.897031
4 -5.748749  1.162952

DataFrame de Teste:
        cp1       cp2
0 -2.686743 -1.216821
1 -4.331255 -0.766327
2 -4.985360  0.371301
3 -5.099876  0.243743
4 -5.023000 -0.518739


In [314]:
significante_variavel_(X_train, 2, top_n=3)

,Component,Variable,Loading
0,PC1,281 fBodyAcc-sma(),0.058571
1,PC1,360 fBodyAccJerk-sma(),0.058568
2,PC1,96 tBodyAccJerk-sma(),0.058538
3,PC2,296 fBodyAcc-meanFreq()-Z,0.122385
4,PC2,249 tBodyGyroMag-arCoeff()1,0.119783
5,PC2,210 tBodyAccMag-arCoeff()1,0.118162


In [318]:
%%time
clf2 = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001).fit(pc_treino, y_train)

CPU times: total: 31.2 ms
Wall time: 36.2 ms


In [321]:
metricas_avaliacao(clf2, pc_treino, y_train, pc_teste, y_test)

{'Acurácia na base de treino': 0.613,
 'Acurácia na base de teste': 0.585,
 'Acurácia média da validação cruzada': 0.574,
 'Desvio padrão da validação cruzada': 0.034}

Utilizando dois componentes eu ainda tenho um desempenho espetacular em relação ao uso de todas as variáveis e já tive um aumento nas métricas de avalição se comparado com apenas 1 componente. Entretanto, essas métricas ainda não são muito satisfatórias, prosseguiremos testanto outros números de componentes. 

#### Transformação do dataframe e obtenção de três componentes componentes:

In [326]:
%%time
#transformação do df para 3 componentes:
prcomp = PCA(n_components=3).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

CPU times: total: 375 ms
Wall time: 127 ms


In [328]:
#exibindo os componentes criados:
df3= pca_qtd(X_train, X_test, n=3)
df3

DataFrame de Treino:
        cp1       cp2       cp3
0 -5.520280 -0.290278 -1.529929
1 -5.535350 -0.082530 -1.924804
2 -5.474988  0.287387 -2.144642
3 -5.677232  0.897031 -2.018220
4 -5.748749  1.162952 -2.139533

DataFrame de Teste:
        cp1       cp2       cp3
0 -2.686743 -1.216821 -0.722075
1 -4.331255 -0.766327 -1.128404
2 -4.985360  0.371301 -1.656858
3 -5.099876  0.243743 -1.802703
4 -5.023000 -0.518739 -1.871078


In [330]:
significante_variavel_(X_train, 3, top_n=3)

,Component,Variable,Loading
0,PC1,281 fBodyAcc-sma(),0.058571
1,PC1,360 fBodyAccJerk-sma(),0.058568
2,PC1,96 tBodyAccJerk-sma(),0.058538
3,PC2,296 fBodyAcc-meanFreq()-Z,0.122385
4,PC2,249 tBodyGyroMag-arCoeff()1,0.119783
5,PC2,210 tBodyAccMag-arCoeff()1,0.118162
6,PC3,57 tGravityAcc-energy()-X,0.110061
7,PC3,"66 tGravityAcc-arCoeff()-X,1",0.109914
8,PC3,64 tGravityAcc-entropy()-Y,0.109555


In [332]:
%%time
clf3 = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001).fit(pc_treino, y_train)

CPU times: total: 62.5 ms
Wall time: 47.5 ms


In [334]:
metricas_avaliacao(clf3, pc_treino, y_train, pc_teste, y_test)

{'Acurácia na base de treino': 0.78,
 'Acurácia na base de teste': 0.754,
 'Acurácia média da validação cruzada': 0.715,
 'Desvio padrão da validação cruzada': 0.037}

Com 3 componentes tivemos um aumento legal na acurácia e o nosso tempo ainda é muito bom, 112 vezes mais rápido se comparado ao modelo performando com todas as variáveis. 

#### Transformação do dataframe e obtenção de cinco componentes componentes:

In [298]:
%%time
#transformação do df para 5 componentes:
prcomp = PCA(n_components=5).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

CPU times: total: 625 ms
Wall time: 134 ms


In [178]:
df5= pca_qtd(X_train, X_test, n=3)
df5

DataFrame de Treino:
        cp1       cp2       cp3
0 -5.520280 -0.290278 -1.529929
1 -5.535350 -0.082530 -1.924804
2 -5.474988  0.287387 -2.144642
3 -5.677232  0.897031 -2.018220
4 -5.748749  1.162952 -2.139533

DataFrame de Teste:
        cp1       cp2       cp3
0 -2.686743 -1.216821 -0.722075
1 -4.331255 -0.766327 -1.128404
2 -4.985360  0.371301 -1.656858
3 -5.099876  0.243743 -1.802703
4 -5.023000 -0.518739 -1.871078


In [288]:
#exibindo os componentes criados:
significante_variavel_(X_train, 5, top_n=3)

,Component,Variable,Loading
0,PC1,281 fBodyAcc-sma(),0.058571
1,PC1,360 fBodyAccJerk-sma(),0.058568
2,PC1,96 tBodyAccJerk-sma(),0.058538
3,PC2,296 fBodyAcc-meanFreq()-Z,0.122385
4,PC2,249 tBodyGyroMag-arCoeff()1,0.119783
5,PC2,210 tBodyAccMag-arCoeff()1,0.118162
6,PC3,57 tGravityAcc-energy()-X,0.110061
7,PC3,"66 tGravityAcc-arCoeff()-X,1",0.109915
8,PC3,64 tGravityAcc-entropy()-Y,0.109555
9,PC4,52 tGravityAcc-max()-Z,0.143971


In [341]:
%%time
clf5 = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001).fit(pc_treino, y_train)

CPU times: total: 31.2 ms
Wall time: 28.5 ms


metricas_avaliacao(clf5, pc_treino, y_train, pc_teste, y_test)


Em relação ao número de componentes anterior, tivemos uma melhora nas métricas, mas também um aumento do desvio padrão mas este continua em um valor legal e o nosso tempo continua muito bom.

________________________________________________________________________________________________________________________________________________________

Continuaremos testando alguns outros números de componentes, mas aqui nos atentaremos apenas as métricas, mas caso sinta curiosidade de visualizar os novos componentes criados ou então as principais variáveis para esses componentes, fique a vontade e utilize a função metricas_avaliacao.

#### Obtenção da acurácia de dez componentes componentes:

In [349]:
%%time
#transformação do df para 10 componentes:
prcomp = PCA(n_components=10).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

CPU times: total: 438 ms
Wall time: 191 ms


In [357]:
%%time
clf10 = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001).fit(pc_treino, y_train)

CPU times: total: 125 ms
Wall time: 101 ms


In [355]:
metricas_avaliacao(clf10, pc_treino, y_train, pc_teste, y_test)

{'Acurácia na base de treino': 0.893,
 'Acurácia na base de teste': 0.824,
 'Acurácia média da validação cruzada': 0.794,
 'Desvio padrão da validação cruzada': 0.045}

Agora nosso tempo é pouco mais que 3 vezes maior que o tempo de processamento para 5 componentes, mas ainda é muito favorável em relação à árvore testada com todas as variáveis. A performance aqui também é boa, sendo superior à anterior.

#### Obtenção da acurácia de cinquenta componentes:

In [302]:
%%time
#transformação do df para 50 componentes:
prcomp = PCA(n_components=50).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

CPU times: total: 844 ms
Wall time: 226 ms


In [365]:
%%time
clf50 = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001).fit(pc_treino, y_train)

CPU times: total: 141 ms
Wall time: 139 ms


In [367]:
metricas_avaliacao(clf50, pc_treino, y_train, pc_teste, y_test)

{'Acurácia na base de treino': 0.893,
 'Acurácia na base de teste': 0.824,
 'Acurácia média da validação cruzada': 0.794,
 'Desvio padrão da validação cruzada': 0.045}

Pudemos perceber aqui que nossa acurácia foi aumentando conforme o número de componentes mas também o nosso tempo. Há uma relação de aumento de complexidade mas também aumento de custo. Obtivemos com 50 componentes um resultado muito satisfatório, uma vez que nosso desvio padrão é baixo e nos garante consistencia nos dados e nossa acurácia de validação cruzada é quase 80%. Perdemos entre 5 e 6% de acurácia sobre o df original mas ganhamos um processamento 38x mais rápido. **Concluímos aqui que há compensação em utilizar a redução de dimensionalidade.**

## Utilizando GridSearch:

Há uma maneira de tornar o processo acima ainda mais rápido, que é utilizando o GridSearch para encontrar o melhor número de componentes. Vejamos:

In [385]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Supondo que você já tenha os dados preparados
X = X_train  # seu dataframe
y = y_train  # seu vetor de classes

# Definindo o pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Normalização dos dados
    ('pca', PCA()),  # PCA será ajustado pelo GridSearch
    ('classifier', DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001))  # Classificador
])

# Definindo o parâmetro para o GridSearch
param_grid = {
    'pca__n_components': [1, 2, 3, 5, 10, 50, 60, 80, 100],  # Números de componentes principais
}

# Configurando o GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Executando o GridSearch
grid_search.fit(X, y)

# Resultados
print("Melhores parâmetros:", grid_search.best_params_)
print("Melhor acurácia:", grid_search.best_score_)


Melhores parâmetros: {'pca__n_components': 60}
Melhor acurácia: 0.7661901524715937


Obtivemos aqui que a melhor árvore é com 60 componentes. Vamos imprimir as métricas:

In [392]:
%%time
#transformação do df para 50 componentes:
prcomp = PCA(n_components=50).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

CPU times: total: 938 ms
Wall time: 209 ms


In [394]:
%%time
clf60 = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001).fit(pc_treino, y_train)

CPU times: total: 391 ms
Wall time: 377 ms


In [396]:
metricas_avaliacao(clf50, pc_treino, y_train, pc_teste, y_test)

{'Acurácia na base de treino': 0.916,
 'Acurácia na base de teste': 0.823,
 'Acurácia média da validação cruzada': 0.785,
 'Desvio padrão da validação cruzada': 0.046}

Resultados também muito satisfatórios. Logo, utilizar a redução de componentes, apesar de diminuir um pouco a acurácia do nosso modelo, traz um custo benefício válido, visto que diminui e muito o tempo de processamento.